In [16]:
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights
from torch.utils.data import DataLoader
from torcheval.metrics import R2Score
import pandas as pd
import torch
from torch.utils.data import random_split
import shutil
import os
from dataset import PGLSDataset
from models import PGLSModel, SimpleTabularModel

In [17]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda


In [18]:
for folder in ["data/train_images", "data/test_images"]:
    if "0" not in os.listdir(folder):
        print("Moving images to 0 folder")
        os.makedirs(f"{folder}/0")
        for filename in os.listdir(folder):
            if filename.lower().endswith(".jpeg"):
                source_path = os.path.join(folder, filename)
                target_path = os.path.join(f"{folder}/0", filename)

                shutil.move(source_path, target_path)

In [19]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize the image to 224x224
    transforms.ToTensor()            # Convert PIL image to tensor (H x W x C) in the range [0.0, 1.0]
])

batch_size = 4

In [20]:
train_images_path = 'data/train_images'
test_images_path = 'data/test_images'

train_csv_path = 'data/train.csv'
test_csv_path = 'data/test.csv'


tabular_data = pd.read_csv(train_csv_path)
targets = ["X4", "X11", "X18", "X26", "X50", "X3112"]

# Filter data
upper_values = {}
for target in targets:
    upper_values[target] = tabular_data[target+"_mean"].quantile(0.99)
    tabular_data = tabular_data[tabular_data[target+"_mean"] < upper_values[target]]
    tabular_data = tabular_data[tabular_data[target+"_mean"] > 0]

# Normalize the targets
original_means = tabular_data[[f"{target}_mean" for target in targets]].mean()
original_stds = tabular_data[[f"{target}_mean" for target in targets]].std()
tabular_data[[f"{target}_mean" for target in targets]] = (tabular_data[[f"{target}_mean" for target in targets]] - original_means) / original_stds

# Normalize the features
tabular_input_size = 0
for column in tabular_data.columns:
    if column in ["id"]+[target+"_mean" for target in targets]+[target+"_sd" for target in targets]:
        continue
    tabular_input_size += 1
    min_val = tabular_data[column].min()
    max_val = tabular_data[column].max()
    tabular_data[column] = (tabular_data[column] - min_val) / (max_val - min_val)


test_tabular_data = pd.read_csv(test_csv_path)
# Normalize the features
for column in test_tabular_data.columns:
    if column in ["id"]:
        continue
    min_val = test_tabular_data[column].min()
    max_val = test_tabular_data[column].max()
    test_tabular_data[column] = (test_tabular_data[column] - min_val) / (max_val - min_val)

train_images_dataset = ImageFolder(root=train_images_path, transform=transform)
test_images_dataset = ImageFolder(root=test_images_path, transform=transform)

train_image_csv_dataset = PGLSDataset(tabular_data=tabular_data, image_folder=train_images_dataset, transform_csv=None)
train, val = random_split(train_image_csv_dataset, [int(0.8*len(train_image_csv_dataset)), len(train_image_csv_dataset) - int(0.8*len(train_image_csv_dataset))])
test_image_csv_dataset = PGLSDataset(tabular_data=test_tabular_data, image_folder=test_images_dataset, transform_csv=None)


train_data_loader = DataLoader(train, batch_size=batch_size, shuffle=True)
val_data_loader = DataLoader(val, batch_size=batch_size, shuffle=True)
test_data_loader = DataLoader(test_image_csv_dataset, batch_size=batch_size, shuffle=False)

In [32]:
original_stds = torch.from_numpy(original_stds.values).float().to(device)
original_means = torch.from_numpy(original_means.values).float().to(device)
def denormalize_targets(targets):
    return targets * original_stds + original_means

In [37]:
effnet = efficientnet_b0(weights=EfficientNet_B0_Weights)
tabular_model = SimpleTabularModel(input_data_len=tabular_input_size)
model = PGLSModel(effnet, tabular_model)
metric = R2Score()
criterion = torch.nn.MSELoss()
torch.cuda.empty_cache()

/home/szaryvip/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [40]:
model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

model.train()
for epoch in range(2):
    print(f"Epoch {epoch}")
    for data in train_data_loader:
        image, features, targets = data
        image = image.to(device)
        features = features.to(device)
        targets = targets.to(device)
        optimizer.zero_grad()
        outputs = model(image, features)
        outputs_denorm = denormalize_targets(outputs)
        targets_denorm = denormalize_targets(targets)
        loss = criterion(outputs, targets)
        metric.update(outputs_denorm, targets_denorm)
        loss.backward()
        optimizer.step()
        print(loss.item())
        print(metric.compute())

Epoch 0
1.5553114414215088
tensor(-0.1723, device='cuda:0')
1.7458369731903076
tensor(-0.1742, device='cuda:0')
1.6123594045639038
tensor(-0.1756, device='cuda:0')
0.8083957433700562
tensor(-0.1760, device='cuda:0')
1.3745969533920288
tensor(-0.1755, device='cuda:0')
2.9476382732391357
tensor(-0.1720, device='cuda:0')
1.6184685230255127
tensor(-0.1738, device='cuda:0')
1.263737678527832
tensor(-0.1745, device='cuda:0')
0.964460015296936
tensor(-0.1758, device='cuda:0')
1.1087892055511475
tensor(-0.1761, device='cuda:0')
0.6975414156913757
tensor(-0.1753, device='cuda:0')
2.152010679244995
tensor(-0.1785, device='cuda:0')
1.0030863285064697
tensor(-0.1791, device='cuda:0')
0.9457761645317078
tensor(-0.1788, device='cuda:0')
0.8785809278488159
tensor(-0.1800, device='cuda:0')
1.2756803035736084
tensor(-0.1806, device='cuda:0')
0.5454398989677429
tensor(-0.1795, device='cuda:0')
1.347793459892273
tensor(-0.1826, device='cuda:0')
1.5428683757781982
tensor(-0.1837, device='cuda:0')
3.844929

KeyboardInterrupt: 

In [25]:
accumulated_loss = 0
iterations = 0
accumulated_r2 = 0
model.to(device)
model.eval()
for data in val_data_loader:
    image, features, targets = data
    image = image.to(device)
    features = features.to(device)
    targets = targets.to(device)
    outputs = model(image, features)
    loss = criterion(outputs, targets)
    accumulated_loss += loss.item()
    iterations += 1
    outputs = outputs.cpu().detach().numpy()
    targets = targets.cpu().detach().numpy()
    outputs_denorm = torch.from_numpy(denormalize_targets(outputs))
    targets_denorm = torch.from_numpy(denormalize_targets(targets))
    metric.update(outputs_denorm, targets_denorm)
    accumulated_r2 += metric.compute().item()
print("Average loss on validation set: ", accumulated_loss/iterations)
print("Average R2 on validation set: ", accumulated_r2/iterations)

Average loss on validation set:  0.869040786021616
Average R2 on validation set:  0.09196601509144779


In [27]:
predictions = []
model.eval()
model.to(device)
for data in test_data_loader:
    image, features, targets = data
    image = image.to(device)
    features = features.to(device)
    targets = targets.to(device)
    outputs = model(image, features)
    outputs = outputs.cpu().detach().numpy()
    outputs_denorm = denormalize_targets(outputs)
    predictions.append(outputs_denorm)
predictions = [item for sublist in predictions for item in sublist]

In [28]:
def save_predictions(predictions, test_csv_dataframe, filename):
    with open(filename, "w") as f:
        f.write("id,X4,X11,X18,X26,X50,X3112\n")
        for pred, id in zip(predictions, test_csv_dataframe["id"]):
            f.write(f"{id},{','.join([str(p) for p in pred])}\n")

In [29]:
save_predictions(predictions, test_tabular_data, "predictions.csv")